In [1]:
(
    1 + 0
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

1

In [2]:
(
    1 + 1
)  # Utility cell. Sometimes "Jupyter" skipps the first cell when running all cells. Especially after a restart.

2

In [3]:
print(f"N{1+2} utility cell running...")

N3 utility cell running...


In [4]:
# Resolve the path to the 'src' folder with helper functions
import config_notebooks

PYTHONPATH: C:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\src


In [5]:
from n01_import import *

os.getcwd():
c:\Users\Admin\Documents\Univer\MASTER THESIS IDEAS\Flavio Pinehro\11_MasterProject\notebooks
input_file_location:
..\data\01_raw
output_file_location:
..\data\02_ingested


In [6]:
# # Resolve the path to the 'src' folder with helper functions
# import config_notebooks
# from importlib import reload

# # Import the module and reload it
# import n01_import_reload

# reload(n01_import_reload)

# # Reload updated functions
# from n01_import_reload import *

## Basic data imports and common transformations

### Importing info about locations (countries)

In [7]:
input_file_name = "location_country.csv"
# reading the csv file, renaming 'iso3_code' to 'country_code'
location_df = pd.read_csv(
    os.path.join(input_file_location, input_file_name),
    usecols=["country_id", "name_short_en", "iso3_code"],
)
location_df.rename(
    columns={"name_short_en": "country_name", "iso3_code": "country_code"}, inplace=True
)

In [8]:
location_df.head()

,country_id,country_name,country_code
0,533,Aruba,ABW
1,4,Afghanistan,AFG
2,24,Angola,AGO
3,660,Anguilla,AIA
4,8,Albania,ALB


In [9]:
df_stats(location_df, "location_df")

['location_df', <class 'pandas.core.frame.DataFrame'>]
shape:
(252, 3)
data types:
country_id       int64
country_name    object
country_code    object
dtype: object
columns:
['country_id', 'country_name', 'country_code']
       country_id
count      252.00
mean       438.84
std        257.78
min          4.00
25%        217.00
50%        437.00
75%        655.25
max        999.00
Total memory usage: 34296 bytes


### Staging locations data

In [10]:
# Save the locations DataFrame to a .dta file
output_file_name = "locations.parquet"
dtype_dict = {"country_code": "str", "country_name": "str"}
# dtype_dict={'country_id': 'category', 'country_code': 'str', 'country_name': 'str'}
save_hs92_parquet_data(
    data=location_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

country_id column initial type:
int64
country_id column final type:
int16
country_code column initial type:
object
country_code column final type:
object


### Importing info about products

In [11]:
input_file_name = "product_hs92.csv"
# , usecols=['product_id', 'code', 'name_short_en', 'product_level', 'top_parent_id', 'product_id_hierarchy'])
# reading the csv file, renaming 'name_short_en' to 'product_name'
product_df = pd.read_csv(os.path.join(input_file_location, input_file_name))
product_df.rename(
    columns={"name_short_en": "product_name", "code": "product_code"}, inplace=True
)

In [12]:
df_stats(product_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(6390, 6)
data types:
product_id               int64
product_code            object
product_name            object
product_level            int64
top_parent_id            int64
product_id_hierarchy    object
dtype: object
columns:
['product_id', 'product_code', 'product_name', 'product_level', 'top_parent_id', 'product_id_hierarchy']
       product_id  product_level  top_parent_id
count    6,390.00       6,390.00       6,390.00
mean     6,180.37           5.54           4.49
std      2,900.14       0.922409           2.67
min      1.000000       1.000000       1.000000
25%      5,247.25           6.00           2.00
50%      6,844.50           6.00           5.00
75%      8,441.75           6.00           6.00
max     10,039.00           6.00          10.00
Total memory usage: 1536439 bytes


In [13]:
product_df.columns

Index(['product_id', 'product_code', 'product_name', 'product_level',
       'top_parent_id', 'product_id_hierarchy'],
      dtype='object')

In [14]:
product_df.head()

,product_id,product_code,product_name,product_level,top_parent_id,product_id_hierarchy
0,1,0,Textiles,1,1,1
1,2,1,Agriculture,1,2,2
2,3,2,Stone,1,3,3
3,4,3,Minerals,1,4,4
4,5,4,Metals,1,5,5


### Staging products data

In [15]:
dtype_dict = {
    "product_id": "int16",
    "top_parent_id": "int16",
    "product_level": "category",
}
categorical_columns = ["product_id", "top_parent_id", "product_level"]

In [16]:
# Save the raw exports Data to a .parquet file
output_file_name = "product_hierarchy.parquet"
save_hs92_parquet_data(
    data=product_df,
    file_location=output_file_location,
    file_name=output_file_name,
    categorical_columns=categorical_columns,
    dtype_dict=dtype_dict,
)

product_id column initial type:
int64
product_id column final type:
int16
top_parent_id column initial type:
int64
top_parent_id column final type:
int16
product_level column initial type:
int64
product_level column final type:
category


In [17]:
categorical_columns = ["product_level"]

input_file_name = "product_hierarchy.parquet"
product_df = read_hs92_parquet_data(
    output_file_location, input_file_name, categorical_columns=categorical_columns
)

product_level column initial type:
int64
product_level column final type:
category


In [18]:
df_stats(product_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(6390, 6)
data types:
product_id                 int16
product_code              object
product_name              object
product_level           category
top_parent_id              int16
product_id_hierarchy      object
dtype: object
columns:
['product_id', 'product_code', 'product_name', 'product_level', 'top_parent_id', 'product_id_hierarchy']
       product_id  top_parent_id
count    6,390.00       6,390.00
mean     6,180.37           4.49
std      2,900.14           2.67
min      1.000000       1.000000
25%      5,247.25           2.00
50%      6,844.50           5.00
75%      8,441.75           6.00
max     10,039.00          10.00
Total memory usage: 1415233 bytes


In [19]:
dtype_dict = {"product_id": "category", "product_name": "category"}
categorical_columns = ["product_id", "product_name"]

In [20]:
# Save the products Data to a .parquet file
output_file_name = "product_id_names.parquet"
save_hs92_parquet_data(
    data=product_df[["product_id", "product_name"]],
    file_location=output_file_location,
    file_name=output_file_name,
    categorical_columns=categorical_columns,
    dtype_dict=dtype_dict,
)

product_id column initial type:
int16
product_id column final type:
category
product_name column initial type:
object
product_name column final type:
category


### Importing info about population

### Importing world bank data: gdp and population

In [21]:
# Reading file API_SP.POP.TOTL_DS2_en_csv_v2_87.csv, was downloaded from https://data.worldbank.org/
input_file_name = "API_SP.POP.TOTL_DS2_en_csv_v2_87.csv"
# reading the csv file, specifying the row with the column names
raw_population_df = pd.read_csv(
    os.path.join(input_file_location, input_file_name), header=2
)

In [22]:
# Reading file API_NY.GDP.MKTP.CD_DS2_en_csv_v2_88.csv, was downloaded from https://data.worldbank.org/
input_file_name = "API_NY.GDP.MKTP.CD_DS2_en_csv_v2_88.csv"
# reading the csv file, specifying the row with the column names
raw_gdp_total_df = pd.read_csv(
    os.path.join(input_file_location, input_file_name), header=2
)

In [23]:
# Reading file API_NY.GDP.PCAP.CD_DS2_en_csv_v2_76.csv, was downloaded from https://data.worldbank.org/
input_file_name = "API_NY.GDP.PCAP.CD_DS2_en_csv_v2_76.csv"
# reading the csv file, specifying the row with the column names
raw_pcap_df = pd.read_csv(os.path.join(input_file_location, input_file_name), header=2)

In [24]:
columns_new = {"Country Code": "country_code", "Country Name": "country_name"}
raw_population_df.rename(columns=columns_new, inplace=True)
raw_gdp_total_df.rename(columns=columns_new, inplace=True)
raw_pcap_df.rename(columns=columns_new, inplace=True)

## Analyzing Population

In [25]:
df_stats(raw_population_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(266, 69)
data types:
country_name       object
country_code       object
Indicator Name     object
Indicator Code     object
1960              float64
                   ...   
2020              float64
2021              float64
2022              float64
2023              float64
Unnamed: 68       float64
Length: 69, dtype: object
columns:
['country_name', 'country_code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'Unnamed: 68']
        

In [26]:
raw_population_df["Indicator Code"].unique()

array(['SP.POP.TOTL'], dtype=object)

In [27]:
raw_population_df["Indicator Code"].value_counts()

Indicator Code
SP.POP.TOTL    266
Name: count, dtype: int64

In [28]:
raw_population_df.head()

,country_name,country_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,"Population, total",SP.POP.TOTL,"54,922.00","55,578.00","56,320.00","57,002.00","57,619.00","58,190.00",...,"107,906.00","108,727.00","108,735.00","108,908.00","109,203.00","108,587.00","107,700.00","107,310.00","107,359.00",NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,"130,072,080.00","133,534,923.00","137,171,659.00","140,945,536.00","144,904,094.00","149,033,472.00",...,"607,123,269.00","623,369,401.00","640,058,741.00","657,801,085.00","675,950,189.00","694,446,100.00","713,090,928.00","731,821,393.00","750,503,764.00",NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,"9,035,043.00","9,214,083.00","9,404,406.00","9,604,487.00","9,814,318.00","10,036,008.00",...,"33,831,764.00","34,700,612.00","35,688,935.00","36,743,039.00","37,856,121.00","39,068,979.00","40,000,412.00","40,578,842.00","41,454,761.00",NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,"97,630,925.00","99,706,674.00","101,854,756.00","104,089,175.00","106,388,440.00","108,772,632.00",...,"418,127,845.00","429,454,743.00","440,882,906.00","452,195,915.00","463,365,429.00","474,569,351.00","485,920,997.00","497,387,180.00","509,398,589.00",NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,"5,231,654.00","5,301,583.00","5,354,310.00","5,408,320.00","5,464,187.00","5,521,981.00",...,"28,157,798.00","29,183,070.00","30,234,839.00","31,297,155.00","32,375,632.00","33,451,132.00","34,532,429.00","35,635,029.00","36,749,906.00",NaN


## Keeping only country code and gdp data

In [29]:
col_years = [*map(str, range(1960, 2024))]

In [30]:
columns = ["country_code"] + col_years

In [31]:
raw_population_df = raw_population_df[columns].reset_index(drop=True)

In [32]:
raw_population_df.head()

,country_code,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,"54,922.00","55,578.00","56,320.00","57,002.00","57,619.00","58,190.00","58,694.00","58,990.00","59,069.00",...,"106,807.00","107,906.00","108,727.00","108,735.00","108,908.00","109,203.00","108,587.00","107,700.00","107,310.00","107,359.00"
1,AFE,"130,072,080.00","133,534,923.00","137,171,659.00","140,945,536.00","144,904,094.00","149,033,472.00","153,281,203.00","157,704,381.00","162,329,396.00",...,"590,968,990.00","607,123,269.00","623,369,401.00","640,058,741.00","657,801,085.00","675,950,189.00","694,446,100.00","713,090,928.00","731,821,393.00","750,503,764.00"
2,AFG,"9,035,043.00","9,214,083.00","9,404,406.00","9,604,487.00","9,814,318.00","10,036,008.00","10,266,395.00","10,505,959.00","10,756,922.00",...,"32,792,523.00","33,831,764.00","34,700,612.00","35,688,935.00","36,743,039.00","37,856,121.00","39,068,979.00","40,000,412.00","40,578,842.00","41,454,761.00"
3,AFW,"97,630,925.00","99,706,674.00","101,854,756.00","104,089,175.00","106,388,440.00","108,772,632.00","111,246,953.00","113,795,019.00","116,444,636.00",...,"406,992,047.00","418,127,845.00","429,454,743.00","440,882,906.00","452,195,915.00","463,365,429.00","474,569,351.00","485,920,997.00","497,387,180.00","509,398,589.00"
4,AGO,"5,231,654.00","5,301,583.00","5,354,310.00","5,408,320.00","5,464,187.00","5,521,981.00","5,581,386.00","5,641,807.00","5,702,699.00",...,"27,160,769.00","28,157,798.00","29,183,070.00","30,234,839.00","31,297,155.00","32,375,632.00","33,451,132.00","34,532,429.00","35,635,029.00","36,749,906.00"


In [33]:
# Reshaping the raw_population_df DataFrame from wide to long format by converting the GDP total data from columns (with years starting from 1960) into rows
population_df = raw_population_df.melt(
    id_vars=["country_code"], var_name="year", value_name="population"
)

In [34]:
# Switching to the 'category' data type for categorical columns
categorical_columns = ["country_code", "year"]
population_df = switch_to_dtype(
    population_df, default_dtype="category", categorical_columns=categorical_columns
)

country_code column initial type:
object
country_code column final type:
category
year column initial type:
object
year column final type:
category


In [35]:
df_stats(population_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code    category
year            category
population       float64
dtype: object
columns:
['country_code', 'year', 'population']
            population
count        16,930.00
mean    216,605,749.67
std     711,507,956.69
min           2,715.00
25%       1,009,540.00
50%       6,748,606.50
75%      46,785,187.00
max   8,061,876,001.00
Total memory usage: 217660 bytes


In [36]:
population_df.head()

,country_code,year,population
0,ABW,1960,"54,922.00"
1,AFE,1960,"130,072,080.00"
2,AFG,1960,"9,035,043.00"
3,AFW,1960,"97,630,925.00"
4,AGO,1960,"5,231,654.00"


In [37]:
# Save the population DataFrame to a .dta file
output_file_name = "raw_population.dta"
dtype_dict = {"country_code": "str"}
save_hs92_stata_data(
    data=population_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

year column initial type:
category
year column final type:
int16
country_code column initial type:
category
country_code column final type:
object


In [38]:
df_stats(population_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code    category
year            category
population       float64
dtype: object
columns:
['country_code', 'year', 'population']
            population
count        16,930.00
mean    216,605,749.67
std     711,507,956.69
min           2,715.00
25%       1,009,540.00
50%       6,748,606.50
75%      46,785,187.00
max   8,061,876,001.00
Total memory usage: 217660 bytes


In [39]:
del raw_population_df
gc.collect()

21

## Analyzing GDP_Total

In [40]:
df_stats(raw_gdp_total_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(266, 69)
data types:
country_name       object
country_code       object
Indicator Name     object
Indicator Code     object
1960              float64
                   ...   
2020              float64
2021              float64
2022              float64
2023              float64
Unnamed: 68       float64
Length: 69, dtype: object
columns:
['country_name', 'country_code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'Unnamed: 68']
        

In [41]:
raw_gdp_total_df["Indicator Code"].unique()

array(['NY.GDP.MKTP.CD'], dtype=object)

In [42]:
raw_gdp_total_df["Indicator Code"].value_counts()

Indicator Code
NY.GDP.MKTP.CD    266
Name: count, dtype: int64

In [43]:
raw_gdp_total_df.head()

,country_name,country_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,"2,962,907,262.57","2,983,635,195.53","3,092,429,050.28","3,276,184,357.54","3,395,798,882.68","2,481,857,122.84","2,929,446,578.25","3,279,343,543.60","3,648,573,136.15",NaN
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,"24,210,632,546.41","24,963,977,550.73","27,078,797,298.77","31,775,749,181.24","30,285,794,056.93","33,813,171,214.32",...,"898,277,807,343.01","828,942,805,457.76","972,998,927,353.68","1,012,306,000,425.25","1,009,721,174,054.91","933,391,782,089.62","1,085,745,178,850.97","1,191,423,176,242.96","1,245,472,471,675.95",NaN
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,"19,134,221,644.73","18,116,572,395.08","18,753,456,497.82","18,053,222,687.41","18,799,444,490.11","19,955,929,052.15","14,259,995,441.08","14,497,243,872.13","17,233,051,620.11",NaN
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,"11,904,954,209.79","12,707,877,772.09","13,630,758,805.13","14,469,094,143.85","15,803,761,887.02","16,921,087,954.57",...,"771,766,914,345.16","694,360,980,069.46","687,849,196,676.20","770,494,988,148.76","826,483,769,985.13","789,801,651,563.29","849,312,422,223.20","883,973,910,898.85","799,105,950,273.83",NaN
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,"90,496,420,506.60","52,761,617,225.93","73,690,154,990.73","79,450,688,259.37","70,897,962,732.03","48,501,561,203.57","66,505,129,987.72","104,399,746,853.40","84,824,654,481.72",NaN


## Keeping only country code and gdp data

In [44]:
col_years = [*map(str, range(1960, 2024))]

In [45]:
columns = ["country_code"] + col_years

In [46]:
raw_gdp_total_df = raw_gdp_total_df[columns].reset_index(drop=True)

In [47]:
raw_gdp_total_df.head()

,country_code,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"2,790,849,720.67","2,962,907,262.57","2,983,635,195.53","3,092,429,050.28","3,276,184,357.54","3,395,798,882.68","2,481,857,122.84","2,929,446,578.25","3,279,343,543.60","3,648,573,136.15"
1,AFE,"24,210,632,546.41","24,963,977,550.73","27,078,797,298.77","31,775,749,181.24","30,285,794,056.93","33,813,171,214.32","36,934,974,916.79","38,452,170,915.87","41,750,716,742.67",...,"978,708,260,612.87","898,277,807,343.01","828,942,805,457.76","972,998,927,353.68","1,012,306,000,425.25","1,009,721,174,054.91","933,391,782,089.62","1,085,745,178,850.97","1,191,423,176,242.96","1,245,472,471,675.95"
2,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"20,497,128,555.70","19,134,221,644.73","18,116,572,395.08","18,753,456,497.82","18,053,222,687.41","18,799,444,490.11","19,955,929,052.15","14,259,995,441.08","14,497,243,872.13","17,233,051,620.11"
3,AFW,"11,904,954,209.79","12,707,877,772.09","13,630,758,805.13","14,469,094,143.85","15,803,761,887.02","16,921,087,954.57","18,034,172,573.26","16,493,747,239.31","17,022,809,177.19",...,"897,415,713,447.02","771,766,914,345.16","694,360,980,069.46","687,849,196,676.20","770,494,988,148.76","826,483,769,985.13","789,801,651,563.29","849,312,422,223.20","883,973,910,898.85","799,105,950,273.83"
4,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"135,966,802,586.71","90,496,420,506.60","52,761,617,225.93","73,690,154,990.73","79,450,688,259.37","70,897,962,732.03","48,501,561,203.57","66,505,129,987.72","104,399,746,853.40","84,824,654,481.72"


In [48]:
# Reshaping the raw_gdp_total_df DataFrame from wide to long format by converting the GDP total data from columns (with years starting from 1960) into rows
gdp_total_df = raw_gdp_total_df.melt(
    id_vars=["country_code"], var_name="year", value_name="gdp_total"
)

In [49]:
# Switching to the 'category' data type for categorical columns
categorical_columns = ["country_code", "year"]
gdp_total_df = switch_to_dtype(
    gdp_total_df, default_dtype="category", categorical_columns=categorical_columns
)

country_code column initial type:
object
country_code column final type:
category
year column initial type:
object
year column final type:
category


In [50]:
df_stats(gdp_total_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code    category
year            category
gdp_total        float64
dtype: object
columns:
['country_code', 'year', 'gdp_total']
                   gdp_total
count              14,307.00
mean    1,184,720,366,607.38
std     5,489,323,660,834.09
min             2,585,955.87
25%         2,293,505,255.48
50%        16,921,087,954.57
75%       201,302,545,054.44
max   106,171,667,873,321.00
Total memory usage: 217660 bytes


In [51]:
gdp_total_df.head()

,country_code,year,gdp_total
0,ABW,1960,NaN
1,AFE,1960,"24,210,632,546.41"
2,AFG,1960,NaN
3,AFW,1960,"11,904,954,209.79"
4,AGO,1960,NaN


In [52]:
# Save the gdp total DataFrame to a .dta file
output_file_name = "raw_gdp_total.dta"
dtype_dict = {"country_code": "str"}
save_hs92_stata_data(
    data=gdp_total_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

year column initial type:
category
year column final type:
int16
country_code column initial type:
category
country_code column final type:
object


In [53]:
df_stats(gdp_total_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code    category
year            category
gdp_total        float64
dtype: object
columns:
['country_code', 'year', 'gdp_total']
                   gdp_total
count              14,307.00
mean    1,184,720,366,607.38
std     5,489,323,660,834.09
min             2,585,955.87
25%         2,293,505,255.48
50%        16,921,087,954.57
75%       201,302,545,054.44
max   106,171,667,873,321.00
Total memory usage: 217660 bytes


In [54]:
del raw_gdp_total_df
gc.collect()

21

## Analyzing raw GDP PCAP (per capita) data

In [55]:
df_stats(raw_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(266, 69)
data types:
country_name       object
country_code       object
Indicator Name     object
Indicator Code     object
1960              float64
                   ...   
2020              float64
2021              float64
2022              float64
2023              float64
Unnamed: 68       float64
Length: 69, dtype: object
columns:
['country_name', 'country_code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'Unnamed: 68']
        

In [56]:
raw_pcap_df["Indicator Code"].unique()

array(['NY.GDP.PCAP.CD'], dtype=object)

In [57]:
raw_pcap_df["Indicator Code"].value_counts()

Indicator Code
NY.GDP.PCAP.CD    266
Name: count, dtype: int64

In [58]:
raw_pcap_df.head()

,country_name,country_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,"27,458.23","27,441.53","28,440.05","30,082.13","31,096.21","22,855.93","27,200.06","30,559.53","33,984.79",NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,186.13,186.95,197.41,225.45,209.01,226.88,...,"1,479.56","1,329.78","1,520.17","1,538.92","1,493.78","1,344.08","1,522.59","1,628.02","1,659.52",NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,565.57,522.08,525.47,491.34,496.60,510.79,356.50,357.26,415.71,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,121.94,127.45,133.83,139.01,148.55,155.56,...,"1,845.77","1,616.84","1,560.16","1,703.90","1,783.65","1,664.25","1,747.84","1,777.24","1,568.72",NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,"3,213.90","1,807.95","2,437.26","2,538.59","2,189.86","1,449.92","1,925.87","2,929.69","2,308.16",NaN


## Analyzing locations data

In [59]:
location_gdp_pcap = raw_pcap_df[["country_code", "country_name"]].sort_values(
    by=["country_code"]
)

In [60]:
location_gdp_pcap.shape

(266, 2)

In [61]:
location_gdp_pcap.head()

,country_code,country_name
0,ABW,Aruba
1,AFE,Africa Eastern and Southern
2,AFG,Afghanistan
3,AFW,Africa Western and Central
4,AGO,Angola


In [62]:
# filtering by the country code 'WLD' (World)
raw_pcap_df[raw_pcap_df["country_code"] == "WLD"]

,country_name,country_code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
259,World,WLD,GDP per capita (current US$),NY.GDP.PCAP.CD,454.06,472.11,497.41,524.35,562.91,600.87,...,"10,142.14","10,188.26","10,731.66","11,288.88","11,334.84","10,916.69","12,352.74","12,737.32","13,169.60",NaN


### Staging locations data (this data we are using later in n06_MOR_visual.ipynb to analyze differences between locations delivered by 'export data' and 'gdp data' files)

In [63]:
# Save the gdp DataFrame to a .dta file
output_file_name = "location_gdp_pcap.dta"
dtype_dict = {"country_code": "str", "country_name": "str"}
save_hs92_stata_data(
    data=location_gdp_pcap,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

country_code column initial type:
object
country_code column final type:
object


## Keeping only country code and gdp pcap data

In [64]:
col_years = [*map(str, range(1960, 2024))]

In [65]:
columns = ["country_code"] + col_years

In [66]:
raw_pcap_df = raw_pcap_df[columns].reset_index(drop=True)

In [67]:
raw_pcap_df.head()

,country_code,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"26,129.84","27,458.23","27,441.53","28,440.05","30,082.13","31,096.21","22,855.93","27,200.06","30,559.53","33,984.79"
1,AFE,186.13,186.95,197.41,225.45,209.01,226.88,240.96,243.82,257.20,...,"1,656.11","1,479.56","1,329.78","1,520.17","1,538.92","1,493.78","1,344.08","1,522.59","1,628.02","1,659.52"
2,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,625.05,565.57,522.08,525.47,491.34,496.60,510.79,356.50,357.26,415.71
3,AFW,121.94,127.45,133.83,139.01,148.55,155.56,162.11,144.94,146.19,...,"2,205.00","1,845.77","1,616.84","1,560.16","1,703.90","1,783.65","1,664.25","1,747.84","1,777.24","1,568.72"
4,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"5,006.00","3,213.90","1,807.95","2,437.26","2,538.59","2,189.86","1,449.92","1,925.87","2,929.69","2,308.16"


In [68]:
# Reshaping the raw_pcap_df DataFrame from wide to long format by converting the GDP per capita data from columns (with years starting from 1960) into rows
gdp_pcap_df = raw_pcap_df.melt(
    id_vars=["country_code"], var_name="year", value_name="gdp_per_capita"
)

In [69]:
# Switching to the 'category' data type for categorical columns
categorical_columns = ["country_code", "year"]
gdp_pcap_df = switch_to_dtype(
    gdp_pcap_df, default_dtype="category", categorical_columns=categorical_columns
)

country_code column initial type:
object
country_code column final type:
category
year column initial type:
object
year column final type:
category


In [70]:
df_stats(gdp_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code      category
year              category
gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita']
       gdp_per_capita
count       14,312.00
mean         8,472.91
std         17,119.11
min             11.80
25%            570.07
50%          1,879.96
75%          7,813.71
max        256,580.52
Total memory usage: 217660 bytes


In [71]:
gdp_pcap_df.head()

,country_code,year,gdp_per_capita
0,ABW,1960,NaN
1,AFE,1960,186.13
2,AFG,1960,NaN
3,AFW,1960,121.94
4,AGO,1960,NaN


In [72]:
# Save the gdp per capita DataFrame to a .dta file
output_file_name = "raw_gdp_pcap.dta"
dtype_dict = {"country_code": "str"}
save_hs92_stata_data(
    data=gdp_pcap_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

year column initial type:
category
year column final type:
int16
country_code column initial type:
category
country_code column final type:
object


In [73]:
df_stats(gdp_pcap_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(17024, 3)
data types:
country_code      category
year              category
gdp_per_capita     float64
dtype: object
columns:
['country_code', 'year', 'gdp_per_capita']
       gdp_per_capita
count       14,312.00
mean         8,472.91
std         17,119.11
min             11.80
25%            570.07
50%          1,879.96
75%          7,813.71
max        256,580.52
Total memory usage: 217660 bytes


In [74]:
del raw_pcap_df
gc.collect()

21

## Processing population data

Here we are going disregard those countries, which on 2010 had population value less than 1250000

In [75]:
population_df.shape

(17024, 3)

In [76]:
# Subsetting the gdp_total_df to only include the 'year' of 2010
population_df = population_df[population_df["year"] == "2010"]

In [77]:
# Subsetting the gdp_total_df to only include the 'country_code' present in the location_df
population_df = population_df[
    population_df["country_code"].isin(location_df["country_code"])
]

In [78]:
population_df.shape

(210, 3)

In [79]:
# sortig the data in descending order of 'population'
population_df_sorted = population_df.sort_values(by=["population"], ascending=False)
population_df_sorted.head()

,country_code,year,population
13340,CHN,2010,"1,337,705,000.00"
13409,IND,2010,"1,243,481,564.00"
13551,USA,2010,"309,327,143.00"
13406,IDN,2010,"246,305,322.00"
13484,PAK,2010,"199,239,047.00"


In [80]:
population_df_sorted.tail()

,country_code,year,population
13528,TCA,2010,"29,739.00"
13555,VGB,2010,"27,675.00"
13488,PLW,2010,"18,396.00"
13545,TUV,2010,"10,614.00"
13479,NRU,2010,"10,043.00"


In [81]:
# Disregarding countries wih 'population' less than 1250000
population_df_high = population_df[population_df["population"] >= 1250000]

# Subset all remaining rows by excluding those in population_df_high
population_df_remaining = population_df[
    ~population_df.index.isin(population_df_high.index)
]

In [82]:
print(population_df_high.shape)
print(population_df_remaining.shape)

(152, 3)
(58, 3)


In [83]:
population_df_high_sorted = population_df_high.sort_values(
    by=["population"], ascending=False
)
population_df_high_sorted.tail()

,country_code,year,population
13500,QAT,2010,"1,616,832.00"
13387,GNB,2010,"1,566,347.00"
13371,EST,2010,"1,331,475.00"
13542,TTO,2010,"1,317,714.00"
13467,MUS,2010,"1,250,400.00"


In [84]:
population_df_high_sorted.head()

,country_code,year,population
13340,CHN,2010,"1,337,705,000.00"
13409,IND,2010,"1,243,481,564.00"
13551,USA,2010,"309,327,143.00"
13406,IDN,2010,"246,305,322.00"
13484,PAK,2010,"199,239,047.00"


In [85]:
population_df_remaining_sorted = population_df_remaining.sort_values(
    by=["population"], ascending=False
)
population_df_remaining_sorted.head(5)

,country_code,year,population
13322,BHR,2010,"1,228,543.00"
13388,GNQ,2010,"1,189,085.00"
13353,CYP,2010,"1,127,816.00"
13524,SWZ,2010,"1,111,650.00"
13537,TLS,2010,"1,084,679.00"


Tangible population countries

In [86]:
pop_countries = set(population_df_high["country_code"])

## Processing 'GDP total' data

In [87]:
# Subsetting the gdp_total_df to only include the 'country_code' present in the location_df
gdp_total_df = gdp_total_df[
    gdp_total_df["country_code"].isin(location_df["country_code"])
]

In [88]:
df_stats(gdp_total_df)

['df', <class 'pandas.core.frame.DataFrame'>]
shape:
(13440, 3)
data types:
country_code    category
year            category
gdp_total        float64
dtype: object
columns:
['country_code', 'year', 'gdp_total']
                  gdp_total
count             11,102.00
mean     190,879,432,598.12
std    1,061,169,753,446.08
min            2,585,955.87
25%        1,382,428,913.62
50%        7,906,951,431.92
75%       51,607,049,459.21
max   27,720,709,000,000.00
Total memory usage: 285624 bytes


Export data is delivered in two initial datasets, covering period 2010-2019. Here we are going disregard those countries, which on 2010 had the 'GDP total' value less than 10<sup>9</sup>$.

In [89]:
# Subsetting the gdp_total_df to only include the 'year' of 2010
gdp_total_df = gdp_total_df[gdp_total_df["year"] == "2010"]

In [90]:
# sortig the data in descending order of 'gdp_total'
gdp_total_df_sorted = gdp_total_df.sort_values(by=["gdp_total"], ascending=False)
gdp_total_df_sorted.head()

,country_code,year,gdp_total
13551,USA,2010,"15,048,971,000,000.00"
13340,CHN,2010,"6,087,191,746,679.49"
13419,JPN,2010,"5,759,071,769,013.11"
13355,DEU,2010,"3,468,154,343,000.01"
13377,FRA,2010,"2,646,230,027,988.34"


In [91]:
gdp_total_df_sorted.tail()

,country_code,year,gdp_total
13479,NRU,2010,"47,562,844.61"
13545,TUV,2010,"32,105,408.48"
13384,GIB,2010,NaN
13493,PRK,2010,NaN
13555,VGB,2010,NaN


In [92]:
# Disregarding countries wih 'gdp_total' less than 1000000000
gdp_total_df_high = gdp_total_df[gdp_total_df["gdp_total"] > 1000000000]

# Subset all remaining rows by excluding those in gdp_total_df_high
gdp_total_df_remaining = gdp_total_df[~gdp_total_df.index.isin(gdp_total_df_high.index)]

In [93]:
print(gdp_total_df_high.shape)
print(gdp_total_df_remaining.shape)

(184, 3)
(26, 3)


In [94]:
gdp_total_df_high_sorted = gdp_total_df_high.sort_values(
    by=["gdp_total"], ascending=False
)
gdp_total_df_high_sorted.tail()

,country_code,year,gdp_total
13369,ERI,2010,"1,589,515,447.15"
13386,GMB,2010,"1,543,294,926.87"
13433,LCA,2010,"1,482,385,185.19"
13312,ATG,2010,"1,298,255,555.56"
13356,DJI,2010,"1,128,611,700.36"


In [95]:
gdp_total_df_remaining_sorted = gdp_total_df_remaining.sort_values(
    by=["gdp_total"], ascending=False
)
gdp_total_df_remaining_sorted.head(5)

,country_code,year,gdp_total
13526,SYC,2010,"981,616,542.33"
13387,GNB,2010,"940,112,538.94"
13346,COM,2010,"907,978,730.89"
13509,SLB,2010,"898,133,684.90"
13525,SXM,2010,"892,290,502.79"


Tangible GDP countries

In [96]:
gdp_countries = set(gdp_total_df_high["country_code"])

## Processing 'GDP per capita' data

### Subsetting to the countries with tangible GDP values

In [97]:
gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["country_code"].isin(gdp_countries)]

In [98]:
# Ordering by year and country_code columns
gdp_pcap_df.sort_values(by=["year", "country_code"], inplace=True)
gdp_pcap_df.reset_index(drop=True, inplace=True)

In [99]:
gdp_pcap_df.head(15)

,country_code,year,gdp_per_capita
0,ABW,1960,NaN
1,AFG,1960,NaN
2,AGO,1960,NaN
3,ALB,1960,NaN
4,AND,1960,NaN
5,ARE,1960,NaN
6,ARG,1960,NaN
7,ARM,1960,NaN
8,ATG,1960,NaN
9,AUS,1960,"1,810.71"


In [100]:
len(set(gdp_pcap_df["country_code"]))

184

In [101]:
"AIA" in set(gdp_pcap_df["country_code"])

False

In [102]:
"WLD" in set(gdp_pcap_df["country_code"])

False

In [103]:
# Calculating number of missing values in 'gdp_per_capita' column
gdp_pcap_df["gdp_per_capita"].isna().sum()

1748

In [104]:
# Searching for the missing values in the merged dataframe in the 'gdp_per_capita' column
missing_gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["gdp_per_capita"].isnull()]

In [105]:
missing_gdp_pcap_df

,country_code,year,gdp_per_capita
0,ABW,1960,NaN
1,AFG,1960,NaN
2,AGO,1960,NaN
3,ALB,1960,NaN
4,AND,1960,NaN
...,...,...,...
11745,SMR,2023,NaN
11748,SSD,2023,NaN
11754,SYR,2023,NaN
11769,VEN,2023,NaN


In [106]:
set(missing_gdp_pcap_df["country_code"])

{'ABW',
 'AFG',
 'AGO',
 'ALB',
 'AND',
 'ARE',
 'ARG',
 'ARM',
 'ATG',
 'AZE',
 'BGR',
 'BHR',
 'BIH',
 'BLR',
 'BRN',
 'BTN',
 'CPV',
 'CUB',
 'CUW',
 'CYM',
 'CYP',
 'CZE',
 'DJI',
 'ERI',
 'EST',
 'FRO',
 'GEO',
 'GIN',
 'GMB',
 'GNQ',
 'GRL',
 'GUM',
 'HRV',
 'HUN',
 'IDN',
 'JOR',
 'KAZ',
 'KGZ',
 'KHM',
 'KWT',
 'LAO',
 'LBN',
 'LCA',
 'LTU',
 'LVA',
 'MAC',
 'MDA',
 'MDV',
 'MKD',
 'MLI',
 'MLT',
 'MNE',
 'MNG',
 'MOZ',
 'MRT',
 'MWI',
 'NAM',
 'NCL',
 'POL',
 'PSE',
 'PYF',
 'QAT',
 'ROU',
 'RUS',
 'SLV',
 'SMR',
 'SRB',
 'SSD',
 'SVK',
 'SVN',
 'SYR',
 'TJK',
 'TKM',
 'TUN',
 'UKR',
 'UZB',
 'VEN',
 'VIR',
 'VNM',
 'YEM'}

In [107]:
# Dropping the missing values in the 'gdp_per_capita' column
gdp_pcap_df.dropna(subset=["gdp_per_capita"], inplace=True)

In [108]:
# Calculating log of 'gdp_per_capita' column
gdp_pcap_df["log_gdp_per_capita"] = np.log(gdp_pcap_df["gdp_per_capita"])

### Subsetting to the countries with tangible population values

In [109]:
gdp_pcap_df = gdp_pcap_df[gdp_pcap_df["country_code"].isin(pop_countries)]

In [110]:
# Ordering by year and country_code columns
gdp_pcap_df.sort_values(by=["year", "country_code"], inplace=True)
gdp_pcap_df.reset_index(drop=True, inplace=True)

In [111]:
gdp_pcap_df.shape

(8334, 4)

In [112]:
gdp_pcap_df.head(15)

,country_code,year,gdp_per_capita,log_gdp_per_capita
0,AUS,1960,"1,810.71",7.50
1,AUT,1960,939.91,6.85
2,BDI,1960,70.91,4.26
3,BEL,1960,"1,290.29",7.16
4,BEN,1960,89.86,4.50
5,BFA,1960,69.15,4.24
6,BGD,1960,82.48,4.41
7,BOL,1960,100.82,4.61
8,BRA,1960,235.27,5.46
9,BWA,1960,58.92,4.08


### Staging GDP data

In [113]:
# Save the clean gdp DataFrame to a .dta file
output_file_name = "ingested_gdp_pcap.dta"
dtype_dict = {"country_code": "str"}
save_hs92_stata_data(
    data=gdp_pcap_df,
    file_location=output_file_location,
    file_name=output_file_name,
    dtype_dict=dtype_dict,
)

year column initial type:
category
year column final type:
int16
country_code column initial type:
category
country_code column final type:
object


Staging final list of countrycodes present in gdp_pcap_df dataframe

In [114]:
valid_gdp_country_codes = set(gdp_pcap_df["country_code"])

In [115]:
# Switching to the 'category' data type for categorical columns
categorical_columns = ["country_id"]
location_df = switch_to_dtype(
    location_df, default_dtype="str", categorical_columns=categorical_columns
)

country_id column initial type:
int64
country_id column final type:
object


In [116]:
# Extracting 'country_id' from the 'location_df' subsetting by 'country_code' present in the 'gdp_pcap_df'
valid_codes_mask = location_df["country_code"].isin(valid_gdp_country_codes)
valid_gdp_country_ids = set(location_df.loc[valid_codes_mask, "country_id"].unique())

In [117]:
valid_gdp_locations = location_df[valid_codes_mask][
    ["country_id", "country_code"]
].reset_index(drop=True)

In [118]:
# Switching to the "int16" data type for merging convenience
categorical_columns = ["country_id"]
valid_gdp_locations = switch_to_dtype(
    valid_gdp_locations, default_dtype="int16", categorical_columns=categorical_columns
)
valid_gdp_locations.dtypes

country_id column initial type:
object
country_id column final type:
int16


country_id       int16
country_code    object
dtype: object

In [119]:
# Making sure I am not removing any vars, which are being staged at later stages
dumped_data = restore_cache(input_file_location=output_file_location)

data restored:['valid_gdp_country_codes', 'valid_gdp_country_ids', 'valid_gdp_locations', 'export_years', 'unreliable_locations', 'rolling_exports_years', 'reliable_gdp_codes']


In [120]:
dumped_data["valid_gdp_country_codes"] = valid_gdp_country_codes
dumped_data["valid_gdp_country_ids"] = valid_gdp_country_ids
dumped_data["valid_gdp_locations"] = valid_gdp_locations

In [121]:
staging_cache(
    output_file_location=output_file_location,
    **dumped_data,
)

data staged:['valid_gdp_country_codes', 'valid_gdp_country_ids', 'valid_gdp_locations', 'export_years', 'unreliable_locations', 'rolling_exports_years', 'reliable_gdp_codes']


In [122]:
# assert False, "End of code"